In [1]:
# coding: utf-8
# наше всё
import numpy as np
import pandas as pd

#настройки pandas, с которыми лучше почти всегда
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

import random

In [2]:
# 1 or 0 under propability
def get_rand(propability):
    import random
    base_random=random.randint(0, 9999)
    
    if float(base_random)<10000*propability:
        res=1
    else:
        res=0
    
    return res

# fuction test
n=100000
prop=0.994

rlst=[]
for i in range(n):
    rlst.append(get_rand(prop))

df=pd.DataFrame(rlst, columns=['rand'])
print('real probabilyhty is ', 100*np.sum(df[df['rand']==1]['rand'])/len(df), '%')

real probabilyhty is  99.406 %


In [3]:
# our citizens
class citizen():
    def __init__(
                 self, 
                 id=-1,
                 age=0,
                 gender='f',
                 parent1_id=-1,
                 parent2_id=-1,
                 couple_id=-1,
                 child_id_lst=[],
                 is_alive=1,
                ):
        global glob_index
        self.id=glob_index+1
        glob_index=glob_index+1
        
        # age by default
        self.age=0
        
        # random gender by default, TO DO: change weights
        # self.gender=random.choice(['f', 'f', 'f', 'm'])
        self.gender=random.choice(['f', 'm'])
                                  
        self.parent1_id=parent1_id
        self.parent2_id=parent2_id
        self.child_id_lst=child_id_lst.copy()
        
        # alive by default
        self.is_alive=1
        
        self.couple_id=couple_id
        
    # print and watch    
    def print_to_df(self):
        df=pd.DataFrame(data=[self.__dict__.values()], columns=self.__dict__.keys())
        return df
    
    # deathing of the citizen    
    def death_case(self, deth_distr_df):
        # there are no zombies
        if self.is_alive==1:
            death_propabilyty=deth_distr_df[
                (deth_distr_df['age']==self.age)&(deth_distr_df['gender']==self.gender)
                                        ]['deth_probability'].values[0]
            self.is_alive=get_rand(1-death_propabilyty)
    
    # finding a couple
    def find_case(self, couple_distr_df, tries):
        # only for alive and single
        if (self.is_alive==1) and (self.couple_id==-1):
            # now only geterosexual pairs, TO DO: realisze other variants
            find_propabilyty=couple_distr_df[
                (couple_distr_df['age']==self.age)&
                  (couple_distr_df['gender']==self.gender)
            ]['couple_probability'].values[0]
            if get_rand(find_propabilyty)==1:
                if self.gender=='f':
                    gender='m'
                else:
                    gender='f'
                # finding loop
                for k in range(tries):
                    # find a random citizen in population
                    candidate_id=find_couple(gender, population)
                    candidate_object=population[candidate_id]

                    # if he/she is single make a couple
                    if (candidate_object.couple_id==-1):
                        # make a couple
                        self.couple_id=candidate_object.id
                        # relize simmetrical operation
                        candidate_object.couple_id=self.id
                        break
                
    # making a new citizen
    def parent_case(self, parent_distr_df):
        if (self.is_alive==1) and (self.couple_id!=-1):
            parent_propabilyty=parent_distr_df[
                (parent_distr_df['age']==self.age)&
                  (parent_distr_df['gender']==self.gender)
            ]['parent_probability'].values[0]
            if get_rand(parent_propabilyty)==1:
                child=citizen(parent1_id=self.id, parent2_id=self.couple_id)
                population.update({child.id: child})
                self.child_id_lst.append(child.id)
                population[self.couple_id].child_id_lst.append(child.id)
                return child

In [4]:
def find_couple(gender, population):
    import random
    candidates=[]
    for i in population.keys():
        if population[i].gender==gender:
            candidates.append(population[i].id)
    return random.choice(candidates)

def population_show(population):
    res_df=pd.DataFrame()
    for i in population.keys():
        b=population[i].print_to_df()
        res_df=pd.concat([res_df, b])
    res_df.index=population.keys()
    return res_df

def main_iteration(population):
    print('deth')
    for i in population.keys():
        population[i].death_case(deth_distr_df)
    
    print('couples')
    for i in population.keys():
        population[i].find_case(couple_distr_df, 3)
    
    print('parent')
    old_population_range=population.copy().keys()
    for i in old_population_range:
        population[i].parent_case(parent_distr_df)
    
    print('age')
    for i in population.keys():
        if population[i].is_alive==1:
            population[i].age=population[i].age+1
        
        
def main_sumulation(population, n_iters):
    print('Start')
    main_df=pd.DataFrame()
    for itr in range(n_iters):
        print('Iter # ', itr, ' starts')
        main_iteration(population)
        itr_df=population_show(population)
        itr_df['period']=itr
        main_df=pd.concat([main_df, itr_df])
    return main_df

In [5]:
# genarate curves 
# TO DO find real data

age_df=pd.DataFrame(range(0, 120))
gender_df=pd.DataFrame(['m', 'f'])

age_df.columns=['age']
gender_df.columns=['gender']


age_df['j']=1
gender_df['j']=1

base_distr_df=age_df.merge(gender_df, on='j')

deth_distr_df=base_distr_df.copy()
deth_distr_df['deth_probability']=0.01

couple_distr_df=base_distr_df.copy()
couple_distr_df['couple_probability']=0.7

parent_distr_df=base_distr_df.copy()
parent_distr_df['parent_probability']=0.1

In [16]:
# generate population
N=20
glob_index=1005

population=dict()
for i in range(N):
    new_citizen=citizen()
    population.update({new_citizen.id: new_citizen})

In [18]:
population_show(population)[0:20]

,id,age,gender,parent1_id,parent2_id,child_id_lst,is_alive,couple_id
1006,1006,0,m,-1,-1,[],1,-1
1007,1007,0,m,-1,-1,[],1,-1
1008,1008,0,f,-1,-1,[],1,-1
1009,1009,0,m,-1,-1,[],1,-1
1010,1010,0,f,-1,-1,[],1,-1
1011,1011,0,f,-1,-1,[],1,-1
1012,1012,0,m,-1,-1,[],1,-1
1013,1013,0,f,-1,-1,[],1,-1
1014,1014,0,f,-1,-1,[],1,-1
1015,1015,0,m,-1,-1,[],1,-1


In [19]:
main_df=main_sumulation(population, 12)

Start
Iter #  0  starts
deth
couples
parent
age
Iter #  1  starts
deth
couples
parent
age
Iter #  2  starts
deth
couples
parent
age
Iter #  3  starts
deth
couples
parent
age
Iter #  4  starts
deth
couples
parent
age
Iter #  5  starts
deth
couples
parent
age
Iter #  6  starts
deth
couples
parent
age
Iter #  7  starts
deth
couples
parent
age
Iter #  8  starts
deth
couples
parent
age
Iter #  9  starts
deth
couples
parent
age
Iter #  10  starts
deth
couples
parent
age
Iter #  11  starts
deth
couples
parent
age


In [20]:
main_df.sample(3)

,id,age,gender,parent1_id,parent2_id,child_id_lst,is_alive,couple_id,period
1021,1021,11,f,-1,-1,[],1,1015,10
1022,1022,2,m,-1,-1,"[1030, 1035]",1,1013,1
1028,1028,1,m,1007,1011,"[1032, 1038, 1043, 1050]",1,-1,2


In [21]:
main_df[main_df['period']==3][0:20]

,id,age,gender,parent1_id,parent2_id,child_id_lst,is_alive,couple_id,period
1006,1006,4,m,-1,-1,"[1027, 1029, 1034, 1046]",1,1020,3
1007,1007,4,m,-1,-1,"[1028, 1037, 1039]",1,1011,3
1008,1008,4,f,-1,-1,"[1053, 1057]",1,-1,3
1009,1009,4,m,-1,-1,[1031],1,1019,3
1010,1010,4,f,-1,-1,"[1026, 1047, 1054]",1,1017,3
1011,1011,4,f,-1,-1,"[1028, 1037, 1039]",1,1007,3
1012,1012,4,m,-1,-1,"[1040, 1048, 1049]",1,1014,3
1013,1013,4,f,-1,-1,"[1030, 1035]",1,1022,3
1014,1014,4,f,-1,-1,"[1040, 1048, 1049]",1,1012,3
1015,1015,4,m,-1,-1,[],1,1021,3


In [22]:
main_df[main_df['id']==1008]

,id,age,gender,parent1_id,parent2_id,child_id_lst,is_alive,couple_id,period
1008,1008,1,f,-1,-1,"[1053, 1057]",1,-1,0
1008,1008,2,f,-1,-1,"[1053, 1057]",1,-1,1
1008,1008,3,f,-1,-1,"[1053, 1057]",1,-1,2
1008,1008,4,f,-1,-1,"[1053, 1057]",1,-1,3
1008,1008,4,f,-1,-1,"[1053, 1057]",0,-1,4
1008,1008,4,f,-1,-1,"[1053, 1057]",0,-1,5
1008,1008,4,f,-1,-1,"[1053, 1057]",0,-1,6
1008,1008,4,f,-1,-1,"[1053, 1057]",0,-1,7
1008,1008,4,f,-1,-1,"[1053, 1057]",0,-1,8
1008,1008,4,f,-1,-1,"[1053, 1057]",0,1039,9


In [23]:
pop_df=population_show(population)
pop_df

,id,age,gender,parent1_id,parent2_id,child_id_lst,is_alive,couple_id
1006,1006,12,m,-1,-1,"[1027, 1029, 1034, 1046]",1,1020
1007,1007,12,m,-1,-1,"[1028, 1037, 1039]",1,1011
1008,1008,4,f,-1,-1,"[1053, 1057]",0,1039
1009,1009,11,m,-1,-1,[1031],0,1019
1010,1010,12,f,-1,-1,"[1026, 1047, 1054]",1,1017
1011,1011,12,f,-1,-1,"[1028, 1037, 1039]",1,1007
1012,1012,12,m,-1,-1,"[1040, 1048, 1049]",1,1014
1013,1013,12,f,-1,-1,"[1030, 1035]",1,1022
1014,1014,12,f,-1,-1,"[1040, 1048, 1049]",1,1012
1015,1015,12,m,-1,-1,[],1,1021


In [24]:
absolute_population=main_df[main_df['is_alive']==1].groupby('period').agg(
                {
                    'id': lambda x: x.nunique(),
                }).reset_index()
absolute_population=absolute_population.rename(columns={'id': 'absolute_population'})

relative_popultion=absolute_population/N
relative_popultion=relative_popultion.rename(columns={'id': 'relative_popultion'})

mortality=main_df[main_df['is_alive']==0].groupby('period').agg(
                {
                    'id': lambda x: x.nunique(),
                }).reset_index()
mortality=mortality.rename(columns={'id': 'mortality'})
mortality['mortality']=mortality['mortality']/N

In [25]:
absolute_population

,period,absolute_population
0,0,20
1,1,21
2,2,24
3,3,26
4,4,29
5,5,31
6,6,35
7,7,38
8,8,39
9,9,40


In [26]:
relative_popultion

,period,absolute_population
0,0.00,1.00
1,0.05,1.05
2,0.10,1.20
3,0.15,1.30
4,0.20,1.45
5,0.25,1.55
6,0.30,1.75
7,0.35,1.90
8,0.40,1.95
9,0.45,2.00


In [27]:
mortality

,period,mortality
0,1,0.05
1,2,0.05
2,3,0.05
3,4,0.10
4,5,0.10
5,6,0.10
6,7,0.10
7,8,0.10
8,9,0.15
9,10,0.15
